In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

file_may = "VR기기 보유자 특성 및 이용 빈도_5월.csv"
file_nov = "VR기기 보유자 특성 및 이용 빈도_11월.csv"

df_may = pd.read_csv(file_may)
df_nov = pd.read_csv(file_nov)

# 데이터 병합 (중복 RESPOND_ID 제거)
df_combined = pd.concat([df_may, df_nov]).drop_duplicates(subset=['RESPOND_ID'])

# 데이터 전처리
# 사용하지 않을 컬럼 제거
df_cluster = df_combined.drop(columns=["RESPOND_ID", "EXAMIN_BEGIN_DE"])

# 결측값 처리 (VR 이용 빈도 결측값을 "0"으로 대체)
df_cluster["VR_MCHN_UTILIIZA_FQ_VALUE"] = df_cluster["VR_MCHN_UTILIIZA_FQ_VALUE"].fillna("0")

# 범주형 변수를 수치형으로 변환 
label_encoders = {}
for col in df_cluster.columns:
    if df_cluster[col].dtype == "object":
        le = LabelEncoder()
        df_cluster[col] = le.fit_transform(df_cluster[col])
        label_encoders[col] = le

# 데이터 정규화 
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

# 최적의 군집 개수 찾기 (Elbow Method)
inertia = []
k_range = range(2, 10)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_scaled)
    inertia.append(kmeans.inertia_)

# Elbow Method 시각화
plt.plot(k_range, inertia, marker='o')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.xlabel('군집 개수')
plt.ylabel('오차 제곱합')
plt.title('최적 k 값 탐색을 위한 엘보우 방법')
plt.show()

# 최적 군집 개수(k) 4개로 결정 
k_optimal = 4
kmeans = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)
clusters = kmeans.fit_predict(df_scaled)

# 군집 결과를 원본 데이터에 추가
df_combined['Cluster'] = clusters

# 군집별 데이터 크기 확인
cluster_counts = df_combined['Cluster'].value_counts()
print(cluster_counts)

# 시각화: 군집별 응답자 수 분포
plt.figure(figsize=(8, 5))
plt.bar(cluster_counts.index, cluster_counts.values, color=['blue', 'skyblue', 'skyblue', 'skyblue'])
plt.xlabel('군집', fontsize=12)
plt.ylabel('응답자 수', fontsize=12)
plt.title('군집별 응답자 수', fontsize=14)
plt.xticks(cluster_counts.index, labels=[f'Cluster {int(c)}' for c in cluster_counts.index])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
#피어슨 상관분석
import pandas as pd

intent_df = pd.read_csv("지능형 기기 이용 의도_VRAR기기.csv", encoding="cp949")
frequency_df = pd.read_csv("지능형 기기 이용빈도_VRAR기기.csv", encoding="cp949")

# 첫 번째 행을 컬럼명으로 지정
intent_df.columns = intent_df.iloc[0]
intent_df = intent_df[1:].reset_index(drop=True)

frequency_df.columns = frequency_df.iloc[0]
frequency_df = frequency_df[1:].reset_index(drop=True)

# 이용 의도 점수 변환 
intent_weights = [1, 2, 3, 6, 0]  # "전혀 없다" ~ "매우 있다", "잘 모르겠다"(0점 처리)
intent_columns = ['전혀 없다 (%)', '별로 없다 (%)', '약간 있다 (%)', '매우 있다 (%)', '잘 모르겠다 (%)']
intent_df['이용 의도 점수'] = (intent_df[intent_columns].astype(float) * intent_weights).sum(axis=1) / 100

# 이용 빈도 점수 변환
frequency_weights = [30, 10, 5, 2, -5, -10]  
frequency_columns = ['거의 매일 (%)', '일주일에 한두 번 (%)', '1개월에 한두 번 (%)', '6개월에 한두 번 (%)', '최근 1년간 이용 한적 없음 (%)', '전혀 이용한 적 없음 (%)']
frequency_df["이용 빈도 점수"] = (frequency_df[frequency_columns].astype(float) * frequency_weights).sum(axis=1) / sum(frequency_weights)

# 필요한 컬럼 선택
intent_df = intent_df[['구분별(1)', '구분별(2)', '이용 의도 점수']]
frequency_df = frequency_df[['구분별(1)', '구분별(2)', '이용 빈도 점수']]

# 병합
merged_df = pd.merge(intent_df, frequency_df, on=['구분별(1)', '구분별(2)'])

# 데이터 타입 변환
merged_df['이용 의도 점수'] = merged_df['이용 의도 점수'].astype(float)
merged_df['이용 빈도 점수'] = merged_df['이용 빈도 점수'].astype(float)

# 그룹별 피어슨 상관계수 계산
group_correlations = {}

unique_groups = merged_df["구분별(1)"].unique()

for group in unique_groups:
    subset = merged_df[merged_df["구분별(1)"] == group]
    if len(subset) > 1:  
        correlation = subset[['이용 의도 점수', '이용 빈도 점수']].corr(method='pearson').iloc[0, 1]
        group_correlations[group] = correlation

# 결과 출력
print(group_correlations)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('가상 현실이 교육에 미치는 영향.csv')
df = pd.DataFrame(data)

df_filtered = df[df['Gender'].isin(['Female', 'Male'])]

plt.figure(figsize=(12, 8))

sns.countplot(x='Field_of_Study', hue='Gender', data=df_filtered,
              palette=['#89CFF0', '#1565C0'])

plt.title('Distribution of Learning Outcomes by Gender and Field of Study')
plt.xlabel('Field of Study')
plt.ylabel('Count of Learning Outcomes')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
rc('font', family='Malgun Gothic')

df_impact = pd.read_csv('impact.csv')
print(df_impact.info())

df_edu = pd.read_csv('education.csv')
print(df_edu.info())

bins = [1, 4, 10]
labels = ['0-4', '5-10']

df_edu['Satisfaction_Range'] = pd.cut(df_edu['Satisfaction_Score'], 
                                      bins=bins, labels=labels, right=True)
satisfaction_grouped = df_edu['Satisfaction_Range'].value_counts().sort_index()

plt.figure(figsize=(10, 6))
satisfaction_grouped.plot(kind='bar', color='skyblue', edgecolor='black')

plt.title('만족도 조사(0점~10점)', fontsize=16)
plt.xlabel('만족 점수 범위', fontsize=12)
plt.ylabel('학생 수', fontsize=12)
plt.xticks(rotation=0, fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# VR 경험 수준과 참여 시간 비교
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plt.rc('font', family='Malgun Gothic')  # Windows 사용자

# 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False  
plt.figure(figsize=(12, 5))
sns.boxplot(x='Prior_VR_Experience', y='Engagement_Time', data=df_edu, palette='coolwarm')
plt.title('VR 경험 수준에 따른 참여 시간')
plt.ylabel('참여 시간(분)')
plt.xlabel('이전의 VR 경험 수준')
plt.show()

# 미래 관심도 분석
plt.figure(figsize=(12, 5))
sns.histplot(df_edu['Future_Interest'], bins=20, kde=True, color='skyblue')
plt.title('VR 교육에서 미래관심의 분포')
plt.xlabel('미래 관심 점수')
plt.ylabel('빈도수')
plt.show()

df_frequency = pd.read_excel('frequency_16-29.xlsx')
print(df_frequency.info())

data = {
    "산업군": ["교육", "게임", "관광", "방송영상", "국방", "의료", "건설", "제조", "오락", "주택", "스포츠", "기타"],
    "비율(%)": [21.1, 17.3, 16.1, 10.3, 9.8, 4.6, 3.0, 4.3, 6.5, 1.2, 2.5, 3.2]
}

df = pd.DataFrame(data)
df_sorted = df.sort_values(by="비율(%)", ascending=False)
colors = ["orange" if industry == "교육" else "skyblue" 
          for industry in df_sorted["산업군"]]

plt.figure(figsize=(10, 6))
plt.bar(df_sorted["산업군"], df_sorted["비율(%)"], color=colors)
plt.xlabel("산업군", fontsize=12)
plt.ylabel("비율 (%)", fontsize=12)
plt.title("VR·AR 적용 예상 산업군 비율", fontsize=14)
plt.xticks(rotation=45, fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.tight_layout()
plt.show()

df_expectation = pd.read_excel("expectation level.xlsx")
print(df_expectation.info())

data = {
    "연령별": ["16-19세", "20-29세", "30-39세", "40-49세", "50-59세", "60세 이상"],
    "평균(점)": [3.5, 3.21, 3.15, 3.06, 3.01, 2.92]
}
df_expectation = pd.DataFrame(data)

plt.figure(figsize=(10, 6))
plt.plot(df_expectation["연령별"], df_expectation["평균(점)"], color="skyblue")

plt.xlabel("연령대", fontsize=12)
plt.ylabel("기대수준", fontsize=12)
plt.title("VR기기에 대한 기대수준(0점~5점)", fontsize=14)
plt.xticks(rotation=45, fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

data = {
    "연도별": ["2020년", "2021년", "2022년", "2023년", "2024년", "2025년", "2026년"],
    "(단위: 억원)": [65605, 73257, 78878, 85140, 92105, 99833, 108319]
}
df_expectation = pd.DataFrame(data)

plt.figure(figsize=(10, 6))
plt.plot(df_expectation["연도별"], df_expectation["(단위: 억원)"], color="purple", 
         marker="o")

for i, value in enumerate(df_expectation["(단위: 억원)"]):
    plt.text(df_expectation["연도별"][i], value + 2000, f"{value:,}", 
             ha='center', fontsize=10, color='black')
    
start_year, end_year = "2020년", "2026년"
start_value = df_expectation["(단위: 억원)"][0]
end_value = df_expectation["(단위: 억원)"].iloc[-1]

plt.annotate(
    "", 
    xy=(end_year, end_value), 
    xytext=(start_year, start_value),
    arrowprops=dict(arrowstyle="->", color="red", lw=2, 
                    linestyle="dotted", connectionstyle="arc3,rad=0.3"),
    fontsize=12, color="red", ha="left"
)

plt.ylim(60000, 115000)
plt.xlabel("연도", fontsize=12)
plt.ylabel("시장 규모 (억원)", fontsize=12)
plt.title("국내 에듀테크 시장 규모 및 전망", fontsize=14)
plt.xticks(rotation=45, fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
vr = pd.read_csv('C:/Users/kelly/OneDrive/바탕 화면/위아이티 프로젝트 데이터 분석(과정3)/사용 csv 파일/VR_대학_교육데이터셋.csv')

#필요한 변수 2개(퀴즈점수, 미래관심)만으로 df 만들기
interest_df = vr[['Quiz_Score', 'Future_Interest']]
interest_df.head()

interest_df.columns = ["퀴즈점수","미래관심"]
#10점 단위로 구간 생성 
bins = range(50,101,10) #50~100까지 10점 단위로 구간 생성
labels = [f"{i}-{i+9}" for i in bins[:-1]] #구간 이름 설정

interest_df = interest_df[interest_df['퀴즈점수'] > 50].copy() #복사본 생성
interest_df['퀴즈점수구간'] = pd.cut(interest_df['퀴즈점수'], bins=bins, labels=labels, right=False)

interest_df

interest_df['퀴즈점수구간'].value_counts().sort_index() #각 퀴즈점수구간별 개수

#퀴즈점수구간별 학생들의 미래관심점수 평균

future_interest_score_mean = (
    interest_df.groupby("퀴즈점수구간", observed=True)["미래관심"]
    .agg("mean")  # '미래관심' 컬럼에 대해 평균 계산
    .reset_index()  # 인덱스를 리셋
    .rename(columns={"미래관심": "미래관심점수평균"})  # 컬럼 이름 변경
)

future_interest_score_mean

import matplotlib.pyplot as plt
import seaborn as sns

# 선형 그래프 생성
plt.figure(figsize=(8, 6))
sns.lineplot(data=future_interest_score_mean, x='퀴즈점수구간', y='미래관심점수평균', marker='o', color='blue')

# 제목과 레이블 설정
plt.title("The average of future interest scores by quiz score section", fontsize=14) #퀴즈점수구간별 미래관심점수 평균
plt.xlabel("Quiz score section", fontsize=12) #퀴즈점수구간
plt.ylabel("Future interest score average", fontsize=12) #미래관심점수 평균

# 그래프 표시
plt.show()